In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'arxiv:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F612177%2F7694881%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240229%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240229T132857Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Db4150aa6cbda2a3b243e138918c772590871977b5f460dc6e840965e651f040a64c4c79960012e21dfafe4d7eac6800777e7037b96e0526bfaf7f4298f0f94884bc3209576d232ff161ca2986c1dbaceb7bf04159341eec89416906c7b4c4398f46035a791c70fb65a7fd884791ab4b336eee08466fdd7deb435342cfa318880f8c093d82199a13ffa71c479b3b5469bb6cdba82646b8beef8cde8535cabdf67497515e7dd1b6a34b89e975bada004e468750e01d8c11e6b453495508daa7147ba4973aac7a6dd9f7353c41c0b42e8cf0bd8e5c2b159f0b928f8d5f8d39cef332416e87b27e8f9348591ea2fb883045dea180fbfe30e1f4cfb0c4a0136f9b444'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


#### Part 2 - Search for ArXiv papers using text vector similarity
Use natural language to search for research papers on ArXiv and<br>
get search results in a format that enables quick review.<br>
by vbookshelf<br>
20 Feb 2024

Part 1 - Build an ArXiv RAG search system w FAISS<br>
https://www.kaggle.com/code/vbookshelf/part-1-build-an-arxiv-rag-search-system-w-faiss

## Introduction

In Part 1 we built a RAG search system that allows us to use natural language to search for ArXiv research papers. In this notebook you can submit search queries and review the results.

We will be running a FAISS exhaustive (brute-force) search. Normally a nearest neigbors search would be used because it's faster. But I found that, even with more that 2.4 million vectors, a FAISS exhaustive search is still very fast.

Here we won't be using OpenAi to generate a natural language output because in this context that feature doesn't add alot of value.

## How to run a search

To run a search you'll need to "Copy and edit" this notebook. Then run each cell.

Please ensure that the GPU (P100) is switched on.

## Install packages

In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 4.9 MB/s eta 0:00:00


In [ ]:
#!pip install faiss-cpu
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 18.8 MB/s eta 0:00:00:00:0100:01


In [ ]:
import pandas as pd
import numpy as np
import os

import json
import re

In [ ]:
# The embeddings and the dataframe created and saved in Part 1

PATH_TO_EMBEDS = '../input/part-1-build-an-arxiv-rag-search-system-w-faiss/compressed_array.npz'
PATH_TO_DF = '../input/part-1-build-an-arxiv-rag-search-system-w-faiss/compressed_dataframe.csv.gz'


In [ ]:
os.listdir('../input/')

['part-1-build-an-arxiv-rag-search-system-w-faiss', 'arxiv']

## Helper functions

In [ ]:
def run_faiss_search(query_text, top_k):

    # Run FAISS exhaustive search

    query = [query_text]

    # Vectorize the query string
    query_embedding = model.encode(query)

    # Run the query
    # index_vals refers to the chunk_list index values
    scores, index_vals = faiss_index.search(query_embedding, top_k)

    # Get the list of index vals
    index_vals_list = index_vals[0]

    return index_vals_list


def run_rerank(index_vals_list, query_text):

    chunk_list = list(df_data['prepared_text'])

    # Replace the chunk index values with the corresponding strings
    pred_strings_list = [chunk_list[item] for item in index_vals_list]

    # Format the input for the cross encoder
    # The input to the cross_encoder is a list of lists
    # [[query_text, pred_text1], [query_text, pred_text2], ...]

    cross_input_list = []

    for item in pred_strings_list:

        new_list = [query_text, item]

        cross_input_list.append(new_list)


    # Put the pred text into a dataframe
    df = pd.DataFrame(cross_input_list, columns=['query_text', 'pred_text'])

    # Save the orginal index (i.e. df_data index values)
    df['original_index'] = index_vals_list

    # Now, score all retrieved passages using the cross_encoder
    cross_scores = cross_encoder.predict(cross_input_list)

    # Add the scores to the dataframe
    df['cross_scores'] = cross_scores

    # Sort the DataFrame in descending order based on the scores
    df_sorted = df.sort_values(by='cross_scores', ascending=False)

    # Reset the index (*This was missed previously*)
    df_sorted = df_sorted.reset_index(drop=True)

    pred_list = []

    for i in range(0,len(df_sorted)):

        text = df_sorted.loc[i, 'pred_text']

        # Get the arxiv id
        # original_index refers to the index values in df_filtered
        original_index = df_sorted.loc[i, 'original_index']
        arxiv_id = df_data.loc[original_index, 'id']
        cat_text = df_data.loc[original_index, 'cat_text']
        title = df_data.loc[original_index, 'title']

        # Crete the link to the research paper pdf
        link_to_pdf = f'https://arxiv.org/pdf/{arxiv_id}'

        item = {
            'arxiv_id': arxiv_id,
            'link_to_pdf': link_to_pdf,
            'cat_text': cat_text,
            'title': title,
            'abstract': text
        }

        pred_list.append(item)

    return pred_list


def print_search_results(pred_list, num_results_to_print):

    for i in range(0,num_results_to_print):

        pred_dict = pred_list[i]

        link_to_pdf = pred_dict['link_to_pdf']
        abstract = pred_dict['abstract']
        cat_text = pred_dict['cat_text']
        title = pred_dict['title']

        print('Title:',title)
        print('Categories:',cat_text)
        print('Abstract:',abstract)
        print('Link to pdf:',link_to_pdf)
        print()


def run_arxiv_search(query_text, num_results_to_print, top_k=300):

    # Run a faiss greedy search
    pred_index_list = run_faiss_search(query_text, top_k)

    # This returns a list of dicts with length equal to top_k
    pred_list = run_rerank(pred_index_list, query_text)

    # Print the results
    print_search_results(pred_list, num_results_to_print)


## Load the embedding vectors and the dataframe

We will load the embeddings and the dataframe from the ouput of the Part 1 notebook.

In [ ]:
# Load the compressed array
embeddings = np.load(PATH_TO_EMBEDS)

# Access the array by the name you specified ('my_array' in this case)
embeddings = embeddings['array_data']

embeddings.shape

(2421966, 384)

In [ ]:
# Load the compressed DataFrame

df_data = pd.read_csv(PATH_TO_DF, compression='gzip')

print(df_data.shape)

#df_data.head()

/tmp/ipykernel_35/3627369456.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_data = pd.read_csv(PATH_TO_DF, compression='gzip')


(2421966, 6)


## Initialize the packages

In [ ]:
# Initialize FAISS

import faiss

embed_length = embeddings.shape[1]

faiss_index = faiss.IndexFlatL2(embed_length)

# Add the embeddings to the index
faiss_index.add(embeddings)

faiss_index.is_trained

True

In [ ]:
# Initialize sentence_transformers

from sentence_transformers import SentenceTransformer

model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Initialize the cross_encoder for reranking

from sentence_transformers import CrossEncoder

# We use a cross-encoder, to re-rank the results list to improve the quality
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# ---------------------------------- #
# RUN A SEARCH

## Run a search

You'll notice that the title is still appended to the abstract. I left it in as a visual check to ensure that I haven't made any errors when displaying the results.

1- You might improve the search results by
providing more details in your search query.<br>
2- I suggest that you enter a similar search query on the ArXiv website to compare the search results and the user experience.<br>
https://arxiv.org/search/advanced

In [ ]:
# *** PLEASE ENTER YOUR SEARCH QUERY HERE ***

query_text = """

privacy exploit on synthetic data

"""


# RUN THE SEARCH
num_results_to_print = 20 # top_k = 300
run_arxiv_search(query_text, num_results_to_print)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Title: TAPAS: a Toolbox for Adversarial Privacy Auditing of Synthetic Data
Categories: Cryptography and Security, Artificial Intelligence, Machine Learning
Abstract: TAPAS: a Toolbox for Adversarial Privacy Auditing of Synthetic Data {title} Personal data collected at scale promises to improve decision-making and accelerate innovation. However, sharing and using such data raises serious privacy concerns. A promising solution is to produce synthetic data, artificial records to share instead of real data. Since synthetic records are not linked to real persons, this intuitively prevents classical re-identification attacks. However, this is insufficient to protect privacy. We here present TAPAS, a toolbox of attacks to evaluate synthetic data privacy under a wide range of scenarios. These attacks include generalizations of prior works and novel attacks. We also introduce a general framework for reasoning about privacy threats to synthetic data and showcase TAPAS on several examples.
Link t